In [4]:

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

load_dotenv()
RESUME_PATH = "data/resume.pdf" 

class ResumeProfile(BaseModel):
    full_name: str = Field(description="The full name of the candidate.")
    
    target_role: str = Field(description="The specific job title the candidate is aiming for based on their summary/bio (e.g. 'Data Scientist', 'Frontend Dev').")
    
    location: str = Field(description="Candidate's city/country. Return 'Remote' if not specified.")
    summary: str = Field(description="A brief summary of the candidate's profile.")
    technical_skills: List[str] = Field(description="List of specific technical tools, languages, and frameworks.")
    experience_years: int = Field(description="Estimated total years of professional experience.")

def analyze_resume(file_path: str) -> ResumeProfile:
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    resume_text = "\n".join([doc.page_content for doc in docs])
    
    print("🧠 Analyzing resume for Target Role & Skills...")
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    structured_llm = llm.with_structured_output(ResumeProfile)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a Career Advisor. Analyze the resume to find the specific role the candidate wants."),
        ("human", "{text}")
    ])
    return (prompt | structured_llm).invoke({"text": resume_text})

if os.path.exists(RESUME_PATH):
    profile = analyze_resume(RESUME_PATH)
    print(f"\n✅ Profile Loaded!")
    print(f"🎯 Target Role: {profile.target_role}") 
    print(f"📍 Location:    {profile.location}")

🧠 Analyzing resume for Target Role & Skills...

✅ Profile Loaded!
🎯 Target Role: UX Designer
📍 Location:    New York, USA


In [5]:
# --- CELL 1: SETUP & FUNCTION DEFINITIONS (TRACED) ---

from tavily import TavilyClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List, Literal, Dict, Any
from urllib.parse import urlparse
import os

# 1. 🛠️ IMPORT TRACEABLE
from langsmith import traceable

# 2. Initialize Clients
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 3. Define Output Data Model
class JobFitAnalysis(BaseModel):
    """The structured output we want from the LLM."""
    match_reasoning: str = Field(description="Internal reasoning: Explain WHY this is a match or mismatch.")
    job_title: str = Field(description="The specific role title.")
    company_name: str = Field(description="Company name. If not found, use domain name.")
    
    matching_skills: List[str] = Field(description="List of skills found in BOTH resume and job.")
    missing_skills: List[str] = Field(description="List of skills found in job but MISSING in resume.")
    
    fit_category: Literal["Perfect Fit", "Good Match", "Irrelevant"] = Field(
        description="Perfect Fit = Role matches & user has most skills. Good Match = Role matches but user needs upskilling."
    )

# 4. Helper: Diversity Filter
def get_diverse_results(results):
    domain_map = {}
    unique_results = []
    for item in results:
        domain = urlparse(item['url']).netloc
        if domain not in domain_map: domain_map[domain] = []
        domain_map[domain].append(item)
    
    max_items = max(len(v) for v in domain_map.values())
    domains = list(domain_map.keys())
    for i in range(max_items):
        for domain in domains:
            if i < len(domain_map[domain]): unique_results.append(domain_map[domain][i])
            
    return unique_results[:6]

# 5. Step 1 Function: Fetch Raw Jobs (NOW TRACED)
@traceable(run_type="tool", name="Tavily Job Search") # <--- ADDED THIS
def step_1_fetch_raw_jobs(profile) -> List[Dict[str, Any]]:
    loc = profile.location if profile.location and profile.location.lower() != "unknown" else "Remote"
    print(f"\n📍 Search Location: {loc} (Press Enter to confirm or type new location)")
    user_loc = input("   > ")
    if user_loc.strip(): loc = user_loc.strip()

    query = f"{profile.target_role} jobs in {loc} hiring now requirements"
    print(f"\n🔍 [Step 1] Fetching raw jobs from Tavily for: '{query}'...")
    
    response = tavily_client.search(
        query=query, 
        topic="general", 
        max_results=12,
        search_depth="advanced",
        include_raw_content=True
    )
    
    raw_results = get_diverse_results(response.get("results", []))
    print(f"✅ [Step 1] Raw search complete. List optimized for diversity.")
    return raw_results

# 6. Step 2 Function: LLM Extraction (NOW TRACED)
@traceable(run_type="chain", name="Extract Job Details") # <--- ADDED THIS
def step_2_extract_details(raw_jobs: List[Dict[str, Any]], profile) -> List[Dict[str, Any]]:
    print(f"\n🧠 [Step 2] Sending unstructured content to LLM for extraction...")
    structured_llm = llm.with_structured_output(JobFitAnalysis)
    processed_jobs = []
    
    for i, job in enumerate(raw_jobs):
        if len(job.get("raw_content", "")) < 200: continue
            
        print(f"   ... Analyzing Job {i+1}...")
        
        prompt_text = f"""
        Analyze this raw job description text.
        
        --- CANDIDATE ---
        Target Role: {profile.target_role}
        Skills: {', '.join(profile.technical_skills)}
        
        --- RAW UNSTRUCTURED JOB TEXT ---
        Title: {job['title']}
        URL: {job['url']}
        CONTENT: {job['raw_content'][:6000]}
        """
        
        try:
            # We don't need @traceable on the LLM call itself because 
            # ChatOpenAI is automatically traced by LangChain!
            prompt = ChatPromptTemplate.from_messages([
                ("system", "You are a Job Data Parser. Convert raw text into structured lists."),
                ("human", "{text}")
            ])
            analysis = (prompt | structured_llm).invoke({"text": prompt_text})
            
            if analysis.fit_category != "Irrelevant":
                job_data = analysis.model_dump()
                job_data['url'] = job['url']
                processed_jobs.append(job_data)
        except Exception as e:
            continue
            
    return processed_jobs

print("✅ Setup Complete. Functions are now traceable.")

✅ Setup Complete. Functions are now traceable.


In [6]:

from IPython.display import display, Markdown

try:
    # 1. Run Step 1 (Fetch)
    # Note: 'profile' variable comes from your Resume Parsing cell (Step 1)
    raw_job_list = step_1_fetch_raw_jobs(profile)
    
    if raw_job_list:
        # 2. Run Step 2 (Extract)
        final_jobs = step_2_extract_details(raw_job_list, profile)
        
        # 3. Display Final Report (Using Markdown for Bold/Clean Text)
        display(Markdown("## 🚀 **Final Job Opportunities Report**"))
        
        if not final_jobs:
            print("❌ No matches found after processing.")
        
        for job in final_jobs:
            # Format lists nicely
            match_str = ", ".join(job['matching_skills']) if job['matching_skills'] else "None"
            miss_str = ", ".join(job['missing_skills']) if job['missing_skills'] else "None"
            
            # Create a Markdown block for each job
            # This allows actual BOLD text and clickable links
            job_card = f"""
### 📌 {job['job_title']} | *{job['company_name']}*
**✅ Matching Skills:** {match_str}  
**⚠️ Missing Skills:** {miss_str}  
🔗 [**Link to Job Application**]({job['url']})
___
"""
            display(Markdown(job_card))
            
    else:
        print("❌ No raw jobs found in Step 1.")

except Exception as e:
    print(f"❌ Execution Error: {e}")


📍 Search Location: New York, USA (Press Enter to confirm or type new location)

🔍 [Step 1] Fetching raw jobs from Tavily for: 'UX Designer jobs in Gurgaon hiring now requirements'...
✅ [Step 1] Raw search complete. List optimized for diversity.

🧠 [Step 2] Sending unstructured content to LLM for extraction...
   ... Analyzing Job 1...
   ... Analyzing Job 2...
   ... Analyzing Job 3...
   ... Analyzing Job 4...
   ... Analyzing Job 5...
   ... Analyzing Job 6...


## 🚀 **Final Job Opportunities Report**


### 📌 Sr. UX Designer | *ixigo*
**✅ Matching Skills:** Adobe Photoshop, Illustrator, Sketch, InVision  
**⚠️ Missing Skills:** User Research, Design Strategy, Data Visualization  
🔗 [**Link to Job Application**](https://designproject.io/jobs/jobs/sr-ux-designer-at-ixigo-2hs253)
___



### 📌 UI/UX Designer | *Destiny HR Group Services*
**✅ Matching Skills:** Adobe Photoshop, Illustrator, Sketch  
**⚠️ Missing Skills:** Figma, Adobe XD, InDesign  
🔗 [**Link to Job Application**](https://www.destinyhrgroup.com/job/jobs-for-ui-ux-designer-in-gurgaon)
___



### 📌 UI/UX Designer | *Systellar Technologies*
**✅ Matching Skills:** Adobe Photoshop, Illustrator, HTML, CSS, JavaScript  
**⚠️ Missing Skills:** Adobe XD, Figma  
🔗 [**Link to Job Application**](https://www.iitjobs.com/job/uiux-designer-gurgaon-haryana-india-systellar-technologies-104667)
___



### 📌 UI/UX Designer | *Programming.com*
**✅ Matching Skills:** Adobe Photoshop, Adobe Illustrator, CSS, JavaScript  
**⚠️ Missing Skills:** Typography, Adobe Creative Suite, Visual design  
🔗 [**Link to Job Application**](https://www.glassdoor.co.in/Job/gurgaon-ui-ux-designer-jobs-SRCH_IL.0,7_IC2921225_KO8,22.htm)
___



### 📌 UX Design Architecture | *apna.co*
**✅ Matching Skills:** Adobe Photoshop, Illustrator, Sketch, InVision, HTML5, CSS, JavaScript  
**⚠️ Missing Skills:** None  
🔗 [**Link to Job Application**](https://apna.co/jobs/dep_ux_design_architecture-jobs-in-sector_25-gurgaon_gurugram)
___



### 📌 Senior UI/ UX Designer | *AKQA*
**✅ Matching Skills:** Sketch, Adobe Photoshop, Illustrator, InVision, HTML5, CSS, JavaScript  
**⚠️ Missing Skills:** Figma, Adobe XD, Framer, Principle, Design systems, Style guides, Modular UI, Component libraries, Usability testing, Rapid prototyping, Iterative design  
🔗 [**Link to Job Application**](https://www.akqa.com/careers/senior-ui-ux-designer-7361729/)
___


In [7]:

from typing import Literal
from collections import Counter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

# 1. Define Advisor Data Models
class RecommendedAction(BaseModel):
    action_type: Literal["Project", "Course", "Certification", "Soft Skill"] = Field(description="Type of recommendation")
    action_name: str = Field(description="Title of the project or course (e.g., 'Build a Redis Clone with Python')")
    reasoning: str = Field(description="Why this specific action helps bridge the gap.")
    priority: Literal["High", "Medium", "Low"] = Field(description="How critical this is for your target role.")

class CareerAdvisorReport(BaseModel):
    summary: str = Field(description="A 2-3 sentence strategic summary of the candidate's standing.")
    top_skill_gaps: List[str] = Field(description="The top 3 most recurring missing skills from the job search.")
    strategic_actions: List[RecommendedAction] = Field(description="3-5 concrete actionable steps to improve employability.")

# 2. Define the Advisor Logic
def generate_career_advice(profile, jobs):
    print(f"\n🧠 Analyzing {len(jobs)} job opportunities for strategic insights...")

    # A. Aggregate Missing Skills
    all_missing = []
    for job in jobs:
        # Check if missing_skills exists and is a list
        if job.get('missing_skills') and isinstance(job['missing_skills'], list):
            all_missing.extend(job['missing_skills'])
    
    # B. Frequency Analysis
    if not all_missing:
        return None
        
    skill_counts = Counter(all_missing)
    # Get top 5 most frequent missing skills
    top_missing = [skill for skill, count in skill_counts.most_common(5)]
    print(f"   ... Identified critical gaps: {', '.join(top_missing)}")

    # C. LLM Analysis
    llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
    advisor_llm = llm.with_structured_output(CareerAdvisorReport)
    
    prompt_text = f"""
    You are a Senior Career Coach. Analyze the candidate's profile against the real-world job market results found.
    
    --- CANDIDATE PROFILE ---
    Target Role: {profile.target_role}
    Current Skills: {', '.join(profile.technical_skills)}
    
    --- MARKET REALITY (JOBS FOUND) ---
    The most frequent missing skills found in job listings are: {', '.join(top_missing)}
    
    --- INSTRUCTIONS ---
    1. Identify the most critical skill gaps blocking "Perfect Fit" roles.
    2. Suggest CONCRETE projects or certifications. 
       - BAD: "Learn Docker"
       - GOOD: "Build a containerized Flask app to demonstrate Docker proficiency."
    3. Be encouraging but realistic.
    """

    try:
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a strategic career advisor. Give actionable, project-based advice."),
            ("human", "{text}")
        ])
        return (prompt | advisor_llm).invoke({"text": prompt_text})
        
    except Exception as e:
        print(f"❌ Advisor Error: {e}")
        return None

# 3. Execution & Display
# Check if we have jobs from the previous step
if 'final_jobs' in locals() and final_jobs:
    report = generate_career_advice(profile, final_jobs)
    
    if report:
        display(Markdown("___"))
        display(Markdown("## 🎓 **Career Advisor Report**"))
        display(Markdown(f"*{report.summary}*"))
        
        display(Markdown("### 🚨 Top Skill Gaps"))
        for skill in report.top_skill_gaps:
            display(Markdown(f"- **{skill}**"))
            
        display(Markdown("### 🛠️ Strategic Action Plan"))
        for action in report.strategic_actions:
            priority_icon = "🔥" if action.priority == "High" else "⚠️"
            card = f"""
#### {priority_icon} {action.action_name} *({action.action_type})*
> "{action.reasoning}"
"""
            display(Markdown(card))
    else:
        print("🎉 No major skill gaps found! You are well-positioned for these roles.")
else:
    print("❌ No job data found. Please run the Job Search (Cell 6) first.")


🧠 Analyzing 6 job opportunities for strategic insights...
   ... Identified critical gaps: Figma, Adobe XD, User Research, Design Strategy, Data Visualization


___

## 🎓 **Career Advisor Report**

*The candidate possesses a solid foundation in UX design tools and front-end web technologies, which are essential for a UX Designer role. However, to align more closely with the current market demands, they need to expand their expertise in newer design tools and methodologies, particularly focusing on Figma, Adobe XD, and user-centered design practices.*

### 🚨 Top Skill Gaps

...


#### 🔥 Redesign a Mobile App using Figma *(Project)*
> "This project will give hands-on experience with Figma, allowing the candidate to demonstrate proficiency in this increasingly popular tool among UX designers."



#### 🔥 Coursera's User Research and Design course *(Course)*
> "User research is key to understanding and meeting user needs. This course will equip the candidate with the necessary skills to conduct effective research, a critical gap in their current skill set."



#### ⚠️ Conduct a Full Design Strategy Workshop *(Project)*
> "Facilitating a workshop on design strategy will improve the candidate's strategic thinking and planning skills, essential for senior-level UX roles."



#### ⚠️ LinkedIn Learning's Adobe XD Essential Training *(Course)*
> "Learning Adobe XD will round out the candidate's toolkit, ensuring they can work with whatever software a prospective employer uses."



#### ⚠️ Create a Data-Driven Dashboard in Adobe XD *(Project)*
> "This project will help in understanding data visualization within UX design, a skill increasingly sought after in the market."


In [1]:
# --- CELL 8: BUILD TRACEABLE LANGGRAPH ---
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, END
from langsmith import traceable
from IPython.display import Image, display

# 1. Define State
class AgentState(TypedDict):
    profile: Any                            # ResumeProfile object
    raw_jobs: List[Dict[str, Any]]          # Output from Tavily Search
    final_jobs: List[Dict[str, Any]]        # Output from LLM Extraction
    career_advice: Optional[Dict[str, Any]] # Output from Career Advisor

# 2. Define Nodes (Wrapping your existing functions)

def node_search(state: AgentState):
    print("\n--- 🟢 NODE: JOB SEARCH ---")
    # Uses your existing traceable function from Cell 5
    raw = step_1_fetch_raw_jobs(state['profile']) 
    return {"raw_jobs": raw}

def node_extract(state: AgentState):
    print("\n--- 🔵 NODE: EXTRACT DETAILS ---")
    # Uses your existing traceable function from Cell 5
    processed = step_2_extract_details(state['raw_jobs'], state['profile'])
    return {"final_jobs": processed}

# We add @traceable here to ensure this specific node step is logged
@traceable(run_type="chain", name="Career Advisor Node")
def node_advisor(state: AgentState):
    print("\n--- 🟣 NODE: CAREER ADVISOR ---")
    # Uses your function from Cell 7
    advice = generate_career_advice(state['profile'], state['final_jobs'])
    
    # Ensure advice is serializable (dict) for the state
    if advice and hasattr(advice, 'dict'):
        advice = advice.dict()
    elif advice and hasattr(advice, 'model_dump'):
        advice = advice.model_dump()
        
    return {"career_advice": advice}

# 3. Build the Graph
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("search_jobs", node_search)
workflow.add_node("analyze_jobs", node_extract)
workflow.add_node("career_advisor", node_advisor)

# Connect Edges (Linear Flow)
workflow.set_entry_point("search_jobs")
workflow.add_edge("search_jobs", "analyze_jobs")
workflow.add_edge("analyze_jobs", "career_advisor")
workflow.add_edge("career_advisor", END)

# Compile
app = workflow.compile()
print("✅ LangGraph Pipeline Built & Traced!")

✅ LangGraph Pipeline Built & Traced!
